## Object Relational Tutorial (1.x API)

The `SQLAlchemy Object Relational Mapper` presents a method of _associating_ __user-defined `Python classes` with `database tables`__, and _instances_ of those classes (_objects_) with `rows` in their corresponding `tables`. It _includes_ a system that __transparently synchronizes all changes in state__ between _objects_ and their _related rows_, called a `unit of work`, as well as a system for _expressing database queries_ in terms of the `user defined classes` and their `defined relationships` between each other.

The `ORM` is in _contrast_ to the `SQLAlchemy Expression Language`, upon which the `ORM` is constructed. Whereas the `SQL Expression Language`, introduced in `SQL Expression Language Tutorial (1.x API)`, presents a system of _representing_ the __primitive constructs of the relational database directly__ _without opinion_, the `ORM` presents a __high level__ and __abstracted__ `pattern of usage`, which itself is an example of applied usage of the _Expression Language_.

While there is __overlap__ among the `usage patterns` of the `ORM` and the `Expression Language`, the __similarities are more superficial__ than they may at first appear. One approaches the _structure and content of data_ from the `perspective` of a __`user-defined domain model`__ which is __transparently persisted and refreshed__ from its _underlying storage model_. The other approaches it from the perspective of __`literal schema` and `SQL expression` representations__ which are __explicitly composed into messages__ _consumed individually by the database_.

A _successful_ `application` may be constructed using the `Object Relational Mapper` __exclusively__. In _advanced situations_, an application constructed with the ORM may make _occasional usage_ of the `Expression Language` __directly__ in certain areas where __specific database interactions are required__.

#### Version Check

A _quick check_ to __verify__ that we are on at `least version 1.4` of `SQLAlchemy`.

In [1]:
import sqlalchemy
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

In [2]:
sqlalchemy.__version__

'1.4.41'

#### Connecting

For this notebook we will use an __in-memory-only__ `SQLite database`. To connect we use `create_engine()`.

In [3]:
engine = create_engine("sqlite:///:memory:", echo=True)

The `echo` flag is a _shortcut_ to __setting up `SQLAlchemy` logging__, which is accomplished via __Python's standard `logging` module__. With it enabled, we'll see _all the generated SQL produced_. If you are working through this notebook and want _less output generated_, set it to `False`. This notebook will _format_ the `SQL` __behind a popup window__ so it doesn't get in our way; just click the `"SQL"` links to see what's being _generated_.

The _return value_ of `create_engine()` is an __instance of `Engine`__, and it _represents_ the __core interface to the database__, adapted through a `dialect` that _handles the details of the database and DBAPI in use_. In this case the `SQLite dialect` will __interpret instructions__ to the Python _built-in_ `sqlite3` module.

> ##### Lazy Connecting
> The `Engine`, when first returned by `create_engine()`, has __not actually tried to connect__ to the database yet; that _happens only the first time it is asked to perform a task_ against the database.

The first time a method like `Engine.execute()` or `Engine.connect()` is called, the `Engine` _establishes_ a __real `DBAPI` connection__ to the database, which is then __used to emit the SQL__. When using the `ORM`, we typically __don't use the `Engine` directly once created__; instead, it's _used behind the scenes_ by the `ORM` as we'll see shortly.

#### Declare a Mapping

When using the `ORM`, the _configurational process_ __starts by `describing` the database tables__ we'll be dealing with, and then by __defining__ our own `classes` which will be _mapped_ to those `tables`. In _modern SQLAlchemy_, these `two tasks` are _usually_ __performed together__, using a system known as `Declarative Extensions`, which allows us to _create classes_ that __include directives to describe__ the `actual database table` they will be _mapped to_.

Classes _mapped_ using the `Declarative system` are _defined_ in terms of a __base class__ which _maintains_ a __catalog of classes and tables relative to that base__ - this is known as the __`declarative base class`__. Our application will _usually_ have __just one instance of this base__ in a _commonly imported module_. We _create_ the `base class` using the `declarative_base()` function, as follows.

In [4]:
Base = declarative_base()

Now that we have a __`"base"`__, we __can define__ _any number of_ `mapped classes` in terms of it. We will start with just a single table called `users`, which will _store records_ for the end-users using our application. A new class called `User` will be the class to which we _map_ this `table`. _Within the class_, we _define_ __details about the table__ to which we'll be __mapping__, _primarily_ the `table name`, and `names` and `datatypes of columns`.

In [5]:
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})>"

> ##### Tip
> The `User` class defines a `__repr__()` method, but note that is __optional__; we only _implement_ it in this notebook so that our examples show _nicely formatted_ `User` _objects_.

A class using `Declarative` at a _minimum_ __needs a `__tablename__` attribute__, and __at least one `Column`__ which is __part of a `primary key`__. `SQLAlchemy` __never makes any assumptions__ by itself _about the table to which a class refers_, including that it has __no built-in conventions__ for _names, datatypes, or constraints_. But this __doesn't mean boilerplate is required__; instead, you're encouraged to _create_ your __own automated conventions__ using `helper functions` and `mixin classes`, which is described in detail at `Mixin and Custom Base Classes`.

When our class is constructed, `Declarative` __replaces all the `Column` objects__ with _special_ `Python accessors` known as __`descriptors`__; this is a process known as __`instrumentation`__. The `"instrumented"` _mapped class_ will provide us with the means to _refer to our table_ in a `SQL context` as well as to __persist and load the values of columns__ from the database.

__Outside of what the `mapping process` does to our class, the class remains otherwise mostly a `normal Python class`__, to which we can _define any number of ordinary attributes and methods_ needed by our application.

#### Create a Schema

With our `User` class _constructed via_ the __Declarative system__, we have _defined information_ about our `table`, known as __`table metadata`__. The object used by `SQLAlchemy` to _represent this information for a specific table_ is called the `Table object`, and here `Declarative` has made one for us. We can see this object by inspecting the `__table__` _attribute_.

In [6]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

> ##### Classical Mappings
> The `Declarative system`, though _highly recommended_, is __not required__ in order to use _SQLAlchemy's ORM_. _Outside of Declarative_, any _plain Python class_ __can be mapped to any `Table`__ using the `mapper()` function __directly__; this _less common usage_ is described at `Imperative Mapping`.

When we declared our class, `Declarative` used a __Python metaclass__ in order to _perform additional activities_ once the class declaration was __complete__; within this phase, it then _created_ a `Table object` __according to our specifications__, and __associated it with the class__ by constructing a `Mapper object`. This object is a __behind-the-scenes__ object we _normally don't need to deal with directly_ (though it can _provide plenty of information_ about our _mapping_ when we need it).

The `Table` object is a _member_ of a _larger collection_ known as __`MetaData`__. When using `Declarative`, this object is _available_ using the `.metadata` attribute of our _declarative base class_.

The `MetaData` is a __registry__ which includes the ability to _emit a limited set of schema generation commands_ to the database. As our `SQLite` database __does not actually have a users table present__, we can use `MetaData` to _issue_ `CREATE TABLE` statements to the database _for all tables that don't yet exist_. Below, we call the `MetaData.create_all()` method, passing in our `Engine` as a __source of database connectivity__. We will see that _special commands are first emitted_ to _check_ for the __presence of the users table__, and following that the __actual `CREATE TABLE` statement__.

In [7]:
Base.metadata.create_all(engine)

2022-10-15 21:33:24,575 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-15 21:33:24,577 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-10-15 21:33:24,578 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-15 21:33:24,580 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-10-15 21:33:24,581 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-15 21:33:24,584 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-15 21:33:24,585 INFO sqlalchemy.engine.Engine [no key 0.00167s] ()
2022-10-15 21:33:24,587 INFO sqlalchemy.engine.Engine COMMIT


> ##### Minimal Table Descriptions vs. Full Descriptions
> 
> Users familiar with the _syntax_ of `CREATE TABLE` may notice that the _VARCHAR columns were generated_ __without a length__; on `SQLite` and `PostgreSQL`, this is a _valid datatype_, but on _others_, it's _not allowed_. So if running this notebook on one of those databases, and you wish to use `SQLAlchemy` to issue `CREATE TABLE`, a `"length"` may be provided to the `String` type as below:
> 
> `Column(String(50))`
> 
> The _length_ field on `String`, as well as _similar_ `precision/scale` fields available on `Integer`, `Numeric`, etc. are __not referenced__ by `SQLAlchemy` other than when _creating tables_.
> 
> Additionally, `Firebird` and `Oracle` __require `sequences` to generate new primary key identifiers__, and `SQLAlchemy` __doesn't generate or assume__ these _without_ being __instructed__. For that, you use the `Sequence` construct.
> 
> ```
> from sqlalchemy import Sequence
> Column(Integer, Sequence("user_id_seq"), primary_key=True)
> ```
> 
> A _full, foolproof_ `Table` generated via our `declarative mapping` is therefore:
> 
> ```
> class User(Base):
>     __tablename__ = "users"
>     id = Column(Integer, Sequence("user_id_seq"), primary_key=True)
>     name = Column(String(50))
>     fullname = Column(String(50))
>     nickname = Column(String(50))
> 
>     def __repr__(self):
>         return "<User(name='%s', fullname='%s', nickname='%s')>" % (
>             self.name,
>             self.fullname,
>             self.nickname,
>         )
> ```
> 
> We include this _more verbose table definition_ __separately__ to __`highlight`__ the _difference between a minimal construct_ __geared__ _primarily_ towards __in-Python usage only__, versus one that will be _used to emit_ `CREATE TABLE` statements on a _particular set of backends_ with __more stringent requirements__.

#### Create an Instance of the Mapped Class

_With mappings complete_, let's now __create and inspect__ a `User` object.

In [8]:
ed_user = User(name="ed", fullname="Ed Jones", nickname="edsnickname")
print(f"{ed_user.name = }, {ed_user.nickname = }, {ed_user.id = }")

ed_user.name = 'ed', ed_user.nickname = 'edsnickname', ed_user.id = None


> ##### the `__init__()` method
> Our `User` class, as defined using the `Declarative system`, has been _provided with a constructor_ (e.g. `__init__()` method) which __automatically accepts keyword names that match the columns we've mapped__. We are _free to define_ any __explicit `__init__()` method__ we prefer on our class, which will __override the default method__ provided by `Declarative`.

Even though we __didn't specify__ it in the _constructor_, the `id` attribute __still produces a value of `None`__ when we _access_ it (_as opposed to Python's usual behavior_ of raising `AttributeError` for an _undefined attribute_). `SQLAlchemy`'s __instrumentation__ normally _produces_ this _default value_ for `column-mapped attributes` __when first accessed__. For those attributes where we've _actually assigned a value_, the _instrumentation system_ is __tracking those assignments__ for use _within_ an eventual `INSERT statement` to be __emitted__ to the database.

#### Creating a Session

We're now ready to start talking to the database. The _ORM_'s `"handle"` to the database is the `Session`. When we _first_ __set up the application__, at the _same level_ as our `create_engine()` statement, we define a `Session` class which will __serve as a factory__ for _new_ `Session` objects.

In [9]:
Session = sessionmaker(bind=engine)

In the case where your _application_ __does not yet have an `Engine`__ when you `define` your _module-level_ objects, just set it up like this.

In [10]:
Session = sessionmaker()

Later, when you _create your engine_ with `create_engine()`, __connect it to the `Session`__ using `sessionmaker.configure()`.

In [11]:
Session.configure(bind=engine)

> ##### Session life-cycle patterns
> The question of when to make a `Session` _depends_ a lot on __what kind of application__ is being built. Keep in mind, the `Session` is just a __workspace for your objects__, _local to a particular database connection_ - if you think of an _application thread_ as a guest at a dinner party, the `Session` is the guest's plate and the objects it holds are the food (and the database...the kitchen?)! More on this topic available at `When do I construct a Session, when do I commit it, and when do I close it?`.

This _custom-made_ `Session` class will __create new Session objects__ which are __bound to our database__. Other __`transactional characteristics`__ may be _defined_ when calling `sessionmaker` as well; these are described in a later chapter. Then, whenever you need to _have a conversation with the database_, you __instantiate__ a `Session`.

In [12]:
session = Session()

The above `Session` is __associated__ with our _SQLite-enabled_ `Engine`, but it __hasn't opened any connections__ yet. When it's _first used_, it __retrieves a connection__ from a __pool of connections__ _maintained by_ the `Engine`, and _holds onto it_ until we __`commit` all changes__ _and/or_ __close__ the `session` object.